In [1]:
from pathlib import Path
import imageio
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Notes
 * Metadata is a temporary dict of stuff that the Axiom code reads from files. 
 * extrinsic and intrinsic parameters are stored in a list of dicts
 * `metadata['coordinate_system']` indicates whether extrinsics are in local `'xyz'` or world `'geo'` coordinates
 
 * `lcp` is a dict lens calibration parameters, a synonym for intrinsics
 * `beta` is an numpy array that contains the values of the extrinsic dict

In [2]:
# These .py files define the objects that load calibration data and do the rectification
%run -i coastcam_funcs.py
%run -i calibration_crs.py
%run -i rectifier_crs.py


In [3]:
image_files       = ['1581508801.c1.timex.jpg','1581508801.c2.timex.jpg']
extrinsic_cal_files = ['CACO01_C1_EOBest.json','CACO01_C2_IOBest.json']
intrinsic_cal_files = ['CACO01_C1_IOBest.json','CACO01_C2_EOBest.json']

In [4]:
metadata= {'name': 'CACO-01', 'serial_number': 1, 'camera_number': 'C1', 'calibration_date': '2019-12-12', 'coordinate_system': 'geo'}
local_origin = {'x': 410935.,'y':4655890., 'angd': 55.}

In [5]:
# read cal files and make lists of cal dicts
extrinsics_list = []
for f in extrinsic_cal_files:
    extrinsics_list.append( json2dict(f) )
intrinsics_list = []
for f in intrinsic_cal_files:
    intrinsics_list.append( json2dict(f) )

In [6]:
# check test for coordinate system
if metadata['coordinate_system'].lower() == 'xyz':
    print('Extrinsics are local coordinates')
elif metadata['coordinate_system'].lower() == 'geo':
    print('Extrinsics are in world coordinates')
else:
    print('Invalid value of coordinate_system: ',metadata['coordinate_system'])
    
print(extrinsics_list[0])
print(extrinsics_list[0]['y']-local_origin['y'])

Extrinsics are in world coordinates
{'x': 410843.97, 'y': 4655942.49, 'z': 27.3, 'a': -0.271, 't': 1.304, 'r': 0.007}
52.49000000022352


In [7]:
calibration = CameraCalibration(metadata,intrinsics_list[0],extrinsics_list[0],local_origin)
print(calibration.local_origin)
print(calibration.world_extrinsics)
print(calibration.local_extrinsics)

{'x': 410935.0, 'y': 4655890.0, 'angd': 55.0}
{'x': 410843.97, 'y': 4655942.49, 'z': 27.3, 'a': -0.271, 't': 1.304, 'r': 0.007}
{'x': -9.21537219613947, 'y': 104.6744377358444, 'z': 27.3, 'a': 0.6889310885968812, 't': 1.304, 'r': 0.007}


In [8]:
xmin = 0.
xmax = 500.
ymin = 0.
ymax = 700.
dx = 2.
dy = 2.
z =  0.

rectifier_grid = TargetGrid(
    [xmin, xmax],
    [ymin, ymax],
    dx,
    dy,
    z
)

rectifier = Rectifier(
    rectifier_grid
)

In [9]:
%%time
%prun rectified_image = rectifier.rectify_images(metadata, image_files, intrinsics_list, extrinsics_list, local_origin)

target_grid.xyz:  (88101, 3)
tmp: (4, 88101)
Shape of UV
(3, 88101)
shape of u and v
(88101,) (88101,)
(88101,)
(2048, 2448, 3) None


ValueError: shape mismatch: objects cannot be broadcast to a single shape

In [10]:
imageio.imwrite('test_image.jpg',rectified_image)

NameError: name 'rectified_image' is not defined

In [ ]:
plt.imshow( rectified_image.astype(np.int))
plt.gca().invert_yaxis()

In [ ]:
print(np.nanmax(rectified_image))